In [1]:
!pip install keras==2.2.5

     |████████████████████████████████| 337kB 3.5MB/s 
  Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


In [2]:
!rm -rf keras-neural-tensor-layer && git clone https://github.com/LanceNorskog/keras-neural-tensor-layer.git
%cd keras-neural-tensor-layer

Cloning into 'keras-neural-tensor-layer'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 41 (delta 0), reused 1 (delta 0), pack-reused 38
Unpacking objects: 100% (41/41), done.
/content/keras-neural-tensor-layer


In [17]:
import math
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from keras import backend as K

from keras.optimizers import SGD, Adam
from keras.layers import Dense

from keras.layers import Input
from keras.models import Model

from neural_tensor_layer import NeuralTensorLayer


def get_data():
  digits = load_digits()
  dig1 = []
  dig2 = []
  label = []
  n = 0
  for i in range(digits.data.shape[0]):
    for j in range(digits.data.shape[0]):
        n = n + 1
        if n % 30 == 0: 
            if digits.target[i] != digits.target[j]:
                dig1.append(digits.data[i])
                dig2.append(digits.data[j])
                if digits.target[i] > digits.target[j]:
                    label.append(1.0)
                else:
                    label.append(0.0)
  count = len(dig1)
  print('samples: ', count)
  (train_i, test_i, _, _) = train_test_split(np.arange(count, dtype='int32'), np.arange(count, dtype='int32'))
  print(train_i[0:10])
  print(test_i[0:10])
  dig1 = np.asarray(dig1, dtype='float32')
  dig2 = np.asarray(dig2, dtype='float32')
  label = np.asarray(label, dtype='float32')
  return dig1[train_i], dig2[train_i], label[train_i], dig1[test_i], dig2[test_i], label[test_i]

X1_train,X2_train,Y_train,X1_test,X2_test,Y_test = get_data()


samples:  96539
[30558 39163 63065 52681 31889 64728 32332 69743 32782 56510]
[31501 13261 44323 18686 14726 46260 70509 26680 26794 11284]


In [19]:

if True:
  input1 = Input(shape=(64,), dtype='float32')
  input2 = Input(shape=(64,), dtype='float32')
  btp = NeuralTensorLayer(output_dim=32, input_dim=64)([input1, input2])

  p = Dense(units=1)(btp)
  model = Model(inputs=[input1, input2], outputs=[p])

  adam = Adam()
  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['binary_accuracy'])
  model.summary()

  history = model.fit([X1_train, X2_train], Y_train, 
            validation_data=[[X1_test, X2_test], Y_test],
            epochs=10, batch_size=32, verbose=2)


Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 64)           0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 64)           0                                            
__________________________________________________________________________________________________
neural_tensor_layer_5 (NeuralTe (None, 32)           135232      input_9[0][0]                    
                                                                 input_10[0][0]                   
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 1)            33          neural_tensor_layer_5[0][0]

In [20]:

plt.figure()
metric_names = ['loss', 'binary_accuracy']
if history != None:
  # summarize history for accuracy
  for m in metric_names:
      #plt.plot(history.history[m])
      plt.plot(history.history['val_' + m])
  plt.title('model accuracy')
  plt.xlabel('epoch')
  sname = []
  for m in metric_names:
      sname.append('{}={:01.3f}'.format(m, history.history['val_' + m][-1]))
  plt.legend(sname, loc='lower right')
  plt.show()

NameError: ignored

In [0]:
if True:
  score = model.evaluate([X1_test, X2_test], Y_test, batch_size=64)
  print('score: ', str(score))
  print()

  #print(K.get_value(model.layers[2].W))
